In [5]:
# Mount drive first
from google.colab import drive
drive.mount('/content/drive')

# List all .360 files in your Drive
!find /content/drive/MyDrive -name "*.360" -type f

Mounted at /content/drive


In [1]:
!pip install opencv-python numpy pytesseract
!apt-get install -y ffmpeg

# Install Tesseract OCR and Python wrapper
!apt-get install -y tesseract-ocr
!pip install pytesseract
!apt-get update
!apt-get install -y ffmpeg libavcodec-extra

# Verify installation
!ffmpeg -version

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 38 not upgraded.
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
tesseract-ocr is already the newest version (4.1.1-2.1build1).
0 upgraded, 0 newly installed, 0 to remove and 38 not upgraded.
Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:2 https://cli.github.com/packages stable InRelease
Get:3 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:6 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:7 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:8 https://developer.download.nvidi

In [6]:
# 3. Set the correct path
video_path = '/content/drive/Shareddrives/Files/360 Videos/Original .360 files/GS012395.360'

# 4. Verify the file exists
import os
if os.path.exists(video_path):
    print(f"✓ File found!")
    print(f"  Size: {os.path.getsize(video_path) / (1024*1024):.2f} MB")
else:
    print(f"✗ File not found at: {video_path}")
    print("\nTrying to list files in the folder:")
    folder_path = '/content/drive/Shareddrives/Files/360 Videos/Original .360 files'
    if os.path.exists(folder_path):
        for file in os.listdir(folder_path):
            print(f"  - {file}")

✗ File not found at: /content/drive/Shareddrives/Files/360 Videos/Original .360 files/GS012395.360

Trying to list files in the folder:


In [3]:
#final cell
import cv2
import os
from pathlib import Path
import matplotlib.pyplot as plt
from IPython.display import display, Image as IPImage
import numpy as np

class WhiteboardDetector:
    """
    Detects and extracts whiteboard regions from video frames.
    Handles various orientations, partial visibility, and filters out false positives.
    """

    def __init__(self, debug=False):
        """
        Initialize the whiteboard detector.

        Args:
            debug: If True, shows visualization at each stage for debugging
        """
        self.debug = debug

        # Configuration parameters for white detection (HSV color space)
        # HSV: Hue, Saturation, Value
        # For white objects: low saturation, high value
        self.white_hsv_lower = np.array([0, 0, 180])      # Lower bound: any hue, low sat, high brightness
        self.white_hsv_upper = np.array([180, 50, 255])   # Upper bound: any hue, low sat, max brightness

        # Shape filtering parameters
        self.min_aspect_ratio = 1.2   # Minimum width:height ratio for whiteboard
        self.max_aspect_ratio = 4.0   # Maximum width:height ratio for whiteboard
        self.min_area = 5000           # Minimum area in pixels (adjust based on your video resolution)
        self.max_area = 500000         # Maximum area in pixels

        # Geometry validation parameters
        self.boundary_threshold = 20   # Pixels from edge to consider "touching boundary"
        self.min_solidity = 0.85       # Minimum solidity (filled vs convex hull)

        # Edge density parameters (Stage 4)
        self.canny_low_threshold = 50    # Lower threshold for Canny edge detection
        self.canny_high_threshold = 150  # Upper threshold for Canny edge detection
        self.min_edge_density = 0.02     # Minimum ratio of edge pixels to total pixels (2%)
                                         # Text on whiteboards creates edges, blank surfaces don't

        # Text verification parameters (Stage 5)
        self.min_text_confidence = 30    # Minimum OCR confidence score (0-100)
        self.min_text_length = 3         # Minimum number of characters detected
        self.require_alphanumeric = True # Require at least some alphanumeric characters

        # Orientation correction parameters (Stage 8)
        self.test_orientations = [0, 90, 180, 270]  # Angles to test for orientation

        # Perspective correction parameters (Stage 7)
        self.roi_padding = 20            # Extra padding around whiteboard when extracting ROI
        self.target_width = 800          # Target width for perspective-corrected whiteboard
        self.target_height = 400         # Target height for perspective-corrected whiteboard

        # Store intermediate results for visualization
        self.white_mask = None
        self.all_contours = []
        self.filtered_contours = []
        self.candidate_boxes = []
        self.edge_density_results = []  # Store edge density info for each candidate
        self.extracted_rois = []        # Store extracted whiteboard ROIs

    def detect_white_regions(self, frame):
        """
        Stage 1: Detect all white regions in the frame using HSV color thresholding.

        Args:
            frame: Input BGR image from video

        Returns:
            binary_mask: Binary mask of white regions
            contours: List of contours found in white regions
        """
        # Convert from BGR (OpenCV default) to HSV color space
        # HSV is better for color-based segmentation than RGB/BGR
        hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

        # Create binary mask: white pixels become 255, others become 0
        # inRange checks if each pixel falls within the HSV bounds
        white_mask = cv2.inRange(hsv, self.white_hsv_lower, self.white_hsv_upper)

        # Morphological operations to clean up the mask
        # 1. Opening: removes small noise (erode then dilate)
        # 2. Closing: fills small holes (dilate then erode)
        kernel = np.ones((5, 5), np.uint8)
        white_mask = cv2.morphologyEx(white_mask, cv2.MORPH_OPEN, kernel, iterations=2)
        white_mask = cv2.morphologyEx(white_mask, cv2.MORPH_CLOSE, kernel, iterations=2)

        # Find contours (boundaries) of white regions
        # RETR_EXTERNAL: only get outermost contours (ignore holes)
        # CHAIN_APPROX_SIMPLE: compress contours to save memory
        contours, _ = cv2.findContours(white_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

        # Store for visualization
        self.white_mask = white_mask
        self.all_contours = contours

        if self.debug:
            print(f"Stage 1: Found {len(contours)} white regions")

        return white_mask, contours

    def filter_by_shape(self, frame, contours):
        """
        Stage 2: Filter contours based on shape characteristics.
        Uses minimum area rectangle to handle rotation and perspective.

        Args:
            frame: Original frame (for getting dimensions)
            contours: List of contours to filter

        Returns:
            filtered_candidates: List of (contour, minAreaRect) tuples that pass filters
        """
        filtered_candidates = []
        frame_height, frame_width = frame.shape[:2]

        for contour in contours:
            # Skip very small contours (noise)
            if len(contour) < 5:  # Need at least 5 points for minAreaRect
                continue

            # Get minimum area rectangle (rotated rectangle that best fits the contour)
            # Returns: ((center_x, center_y), (width, height), angle)
            # This handles rotation automatically!
            rect = cv2.minAreaRect(contour)
            (center_x, center_y), (width, height), angle = rect

            # Ensure width is always the longer dimension for consistent aspect ratio
            if width < height:
                width, height = height, width

            # Calculate area of the bounding box
            area = width * height

            # Filter 1: Area must be within reasonable range
            if area < self.min_area or area > self.max_area:
                if self.debug:
                    print(f"  Rejected: area {area:.0f} outside range [{self.min_area}, {self.max_area}]")
                continue

            # Filter 2: Aspect ratio must match typical whiteboard proportions
            aspect_ratio = width / height if height > 0 else 0
            if aspect_ratio < self.min_aspect_ratio or aspect_ratio > self.max_aspect_ratio:
                if self.debug:
                    print(f"  Rejected: aspect ratio {aspect_ratio:.2f} outside range [{self.min_aspect_ratio}, {self.max_aspect_ratio}]")
                continue

            # Passed all filters!
            filtered_candidates.append((contour, rect))

            if self.debug:
                print(f"  Accepted: area={area:.0f}, aspect_ratio={aspect_ratio:.2f}, angle={angle:.1f}°")

        # Store for visualization
        self.filtered_contours = [c[0] for c in filtered_candidates]
        self.candidate_boxes = [c[1] for c in filtered_candidates]

        if self.debug:
            print(f"Stage 2: {len(filtered_candidates)} candidates passed shape filtering")

        return filtered_candidates

    def validate_shape_geometry(self, frame, contour, rect):
        """
        Stage 3: Validate geometry of candidate shapes.
        Handles partial whiteboards (touching frame boundary) and checks solidity.

        Args:
            frame: Original frame (for boundary checking)
            contour: Contour to validate
            rect: minAreaRect of the contour

        Returns:
            is_valid: Boolean indicating if shape passes validation
            validation_info: Dictionary with validation details
        """
        frame_height, frame_width = frame.shape[:2]

        # Approximate contour to polygon
        # epsilon: maximum distance from contour to approximated polygon
        # Using 2% of perimeter as tolerance
        epsilon = 0.02 * cv2.arcLength(contour, True)
        approx_polygon = cv2.approxPolyDP(contour, epsilon, True)

        num_vertices = len(approx_polygon)

        # Check if any vertices are near the image boundary
        vertices_on_boundary = 0
        for point in approx_polygon:
            x, y = point[0]
            if (x <= self.boundary_threshold or
                x >= frame_width - self.boundary_threshold or
                y <= self.boundary_threshold or
                y >= frame_height - self.boundary_threshold):
                vertices_on_boundary += 1

        # Determine if this is a partial whiteboard (cut off by frame edge)
        is_partial = vertices_on_boundary > 0

        # Validate vertex count
        # Complete whiteboard: should have 4 vertices
        # Partial whiteboard: may have 3-6 vertices (depending on how it's cut)
        valid_vertex_count = False
        if is_partial and 3 <= num_vertices <= 6:
            valid_vertex_count = True
        elif not is_partial and num_vertices == 4:
            valid_vertex_count = True

        # Check convexity (shape should not have indentations)
        is_convex = cv2.isContourConvex(approx_polygon)

        # Calculate solidity: ratio of contour area to its convex hull area
        # High solidity (close to 1.0) means it's a filled shape, not scattered pixels
        contour_area = cv2.contourArea(contour)
        hull = cv2.convexHull(contour)
        hull_area = cv2.contourArea(hull)
        solidity = contour_area / hull_area if hull_area > 0 else 0

        # Overall validation
        is_valid = (valid_vertex_count and
                   (is_convex or is_partial) and  # Allow non-convex if partial
                   solidity >= self.min_solidity)

        validation_info = {
            'num_vertices': num_vertices,
            'is_partial': is_partial,
            'vertices_on_boundary': vertices_on_boundary,
            'is_convex': is_convex,
            'solidity': solidity,
            'valid_vertex_count': valid_vertex_count
        }

        if self.debug:
            print(f"  Geometry validation:")
            print(f"    Vertices: {num_vertices} (on boundary: {vertices_on_boundary})")
            print(f"    Partial: {is_partial}, Convex: {is_convex}, Solidity: {solidity:.2f}")
            print(f"    Result: {'PASS' if is_valid else 'FAIL'}")

        return is_valid, validation_info

    def calculate_edge_density(self, frame, contour, rect):
        """
        Stage 4: Calculate edge density within a candidate region.
        Whiteboards with text have high edge density, blank surfaces have low edge density.

        Args:
            frame: Original frame
            contour: Contour of the candidate region
            rect: minAreaRect of the contour

        Returns:
            edge_density: Ratio of edge pixels to total pixels in region
            edge_info: Dictionary with detailed edge detection information
        """
        # Create a mask for this specific contour
        mask = np.zeros(frame.shape[:2], dtype=np.uint8)
        cv2.drawContours(mask, [contour], -1, 255, -1)  # Fill the contour

        # Convert frame to grayscale for edge detection
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        # Apply Canny edge detection on the entire frame
        # Canny detects edges based on gradient changes
        edges = cv2.Canny(gray, self.canny_low_threshold, self.canny_high_threshold)

        # Apply the mask to get edges only within the candidate region
        edges_in_region = cv2.bitwise_and(edges, edges, mask=mask)

        # Count edge pixels and total pixels in the masked region
        edge_pixel_count = np.count_nonzero(edges_in_region)
        total_pixel_count = np.count_nonzero(mask)

        # Calculate edge density (ratio of edge pixels to total pixels)
        edge_density = edge_pixel_count / total_pixel_count if total_pixel_count > 0 else 0

        # Determine if this passes the edge density threshold
        passes_threshold = edge_density >= self.min_edge_density

        edge_info = {
            'edge_pixel_count': edge_pixel_count,
            'total_pixel_count': total_pixel_count,
            'edge_density': edge_density,
            'passes_threshold': passes_threshold,
            'edges_image': edges_in_region  # Store for visualization
        }

        if self.debug:
            print(f"  Edge density analysis:")
            print(f"    Edge pixels: {edge_pixel_count}")
            print(f"    Total pixels: {total_pixel_count}")
            print(f"    Edge density: {edge_density:.4f} ({edge_density*100:.2f}%)")
            print(f"    Threshold: {self.min_edge_density:.4f} ({self.min_edge_density*100:.2f}%)")
            print(f"    Result: {'PASS' if passes_threshold else 'FAIL'}")

        return edge_density, edge_info

    def extract_whiteboard_roi(self, frame, contour, rect, validation_info):
        """
        Stage 7: Extract and perspective-correct the whiteboard region.
        Handles both complete and partial whiteboards.

        Args:
            frame: Original frame
            contour: Contour of the whiteboard
            rect: minAreaRect of the whiteboard
            validation_info: Geometry validation information

        Returns:
            corrected_roi: Perspective-corrected whiteboard image (frontal view)
            roi_info: Dictionary with extraction information
        """
        frame_height, frame_width = frame.shape[:2]

        # Get the 4 corners of the minimum area rectangle
        # boxPoints returns corners in order, but we need them ordered properly
        box = cv2.boxPoints(rect)
        box = np.float32(box)

        # Sort the points to get them in a consistent order:
        # [top-left, top-right, bottom-right, bottom-left]
        # Method: sort by y-coordinate, then by x-coordinate
        box = sorted(box, key=lambda p: (p[1], p[0]))

        # Split into top and bottom pairs
        top_points = sorted(box[:2], key=lambda p: p[0])  # Top 2, sorted left to right
        bottom_points = sorted(box[2:], key=lambda p: p[0])  # Bottom 2, sorted left to right

        # Order: top-left, top-right, bottom-right, bottom-left
        src_points = np.array([
            top_points[0],      # top-left
            top_points[1],      # top-right
            bottom_points[1],   # bottom-right
            bottom_points[0]    # bottom-left
        ], dtype=np.float32)

        # Handle partial whiteboards (touching frame boundary)
        is_partial = validation_info.get('is_partial', False)

        if is_partial:
            if self.debug:
                print(f"  Detected partial whiteboard - using bounding rectangle")

            # For partial whiteboards, we can't do perfect perspective correction
            # Instead, get the bounding rectangle and extract that region
            x, y, w, h = cv2.boundingRect(contour)

            # Add padding but ensure we stay within frame boundaries
            x1 = max(0, x - self.roi_padding)
            y1 = max(0, y - self.roi_padding)
            x2 = min(frame_width, x + w + self.roi_padding)
            y2 = min(frame_height, y + h + self.roi_padding)

            # Extract the ROI directly (no perspective transform for partial)
            corrected_roi = frame[y1:y2, x1:x2].copy()

            roi_info = {
                'is_partial': True,
                'method': 'bounding_rectangle',
                'original_bbox': (x, y, w, h),
                'padded_bbox': (x1, y1, x2-x1, y2-y1),
                'roi_shape': corrected_roi.shape
            }

        else:
            # Complete whiteboard - apply perspective transform
            if self.debug:
                print(f"  Detected complete whiteboard - applying perspective correction")

            # Calculate the width and height of the whiteboard
            # Use the distances between corner points
            width_top = np.linalg.norm(src_points[1] - src_points[0])
            width_bottom = np.linalg.norm(src_points[2] - src_points[3])
            width = int(max(width_top, width_bottom))

            height_left = np.linalg.norm(src_points[3] - src_points[0])
            height_right = np.linalg.norm(src_points[2] - src_points[1])
            height = int(max(height_left, height_right))

            # Optionally scale to target dimensions while maintaining aspect ratio
            aspect_ratio = width / height if height > 0 else 1
            if width > self.target_width:
                width = self.target_width
                height = int(width / aspect_ratio)

            # Define destination points for frontal view (rectangle)
            dst_points = np.array([
                [0, 0],                    # top-left
                [width - 1, 0],            # top-right
                [width - 1, height - 1],   # bottom-right
                [0, height - 1]            # bottom-left
            ], dtype=np.float32)

            # Compute the perspective transform matrix
            transform_matrix = cv2.getPerspectiveTransform(src_points, dst_points)

            # Apply the perspective transformation
            corrected_roi = cv2.warpPerspective(frame, transform_matrix, (width, height))

            roi_info = {
                'is_partial': False,
                'method': 'perspective_transform',
                'src_points': src_points.tolist(),
                'dst_points': dst_points.tolist(),
                'original_size': (width, height),
                'roi_shape': corrected_roi.shape,
                'transform_matrix': transform_matrix
            }

        if self.debug:
            print(f"  ROI extracted:")
            print(f"    Method: {roi_info['method']}")
            print(f"    Shape: {roi_info['roi_shape']}")
            print(f"    Partial: {is_partial}")

        return corrected_roi, roi_info

    def verify_text_presence(self, roi):
        """
        Stage 5: Verify that the ROI contains readable text using OCR.
        This filters out blank white surfaces and random white objects.

        Args:
            roi: Extracted region of interest (whiteboard candidate)

        Returns:
            has_text: Boolean indicating if meaningful text was found
            text_info: Dictionary with OCR results and details
        """
        try:
            import pytesseract
        except ImportError:
            if self.debug:
                print("  Warning: pytesseract not installed. Skipping text verification.")
                print("  Install with: !pip install pytesseract")
            return True, {'skipped': True, 'reason': 'pytesseract not available'}

        # Convert to grayscale for better OCR
        gray = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)

        # Apply preprocessing to improve OCR accuracy
        # 1. Apply adaptive thresholding to handle varying lighting
        binary = cv2.adaptiveThreshold(
            gray, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
            cv2.THRESH_BINARY, 11, 2
        )

        # 2. Optional: denoise
        denoised = cv2.fastNlMeansDenoising(binary, None, 10, 7, 21)

        # Run OCR with detailed output including confidence scores
        # PSM 6: Assume a single uniform block of text
        # PSM 11: Sparse text - find as much text as possible in no particular order
        try:
            ocr_data = pytesseract.image_to_data(
                denoised,
                output_type=pytesseract.Output.DICT,
                config='--psm 6'  # Try single block first
            )
        except Exception as e:
            if self.debug:
                print(f"  OCR Error: {e}")
            return False, {'error': str(e)}

        # Extract detected text and confidence scores
        detected_text = []
        confidences = []

        for i, conf in enumerate(ocr_data['conf']):
            # Filter out low confidence detections (-1 means no text detected)
            if int(conf) > 0:
                text = ocr_data['text'][i].strip()
                if text:  # Not empty
                    detected_text.append(text)
                    confidences.append(int(conf))

        # Combine all detected text
        full_text = ' '.join(detected_text)

        # Calculate metrics
        num_chars = len(full_text.replace(' ', ''))
        num_alphanumeric = sum(c.isalnum() for c in full_text)
        avg_confidence = sum(confidences) / len(confidences) if confidences else 0

        # Determine if this passes text verification
        has_sufficient_length = num_chars >= self.min_text_length
        has_sufficient_confidence = avg_confidence >= self.min_text_confidence
        has_alphanumeric = num_alphanumeric > 0 if self.require_alphanumeric else True

        passes_verification = (has_sufficient_length and
                              has_sufficient_confidence and
                              has_alphanumeric)

        text_info = {
            'detected_text': full_text,
            'num_characters': num_chars,
            'num_alphanumeric': num_alphanumeric,
            'avg_confidence': avg_confidence,
            'individual_confidences': confidences,
            'passes_verification': passes_verification,
            'has_sufficient_length': has_sufficient_length,
            'has_sufficient_confidence': has_sufficient_confidence,
            'has_alphanumeric': has_alphanumeric
        }

        if self.debug:
            print(f"  Text verification:")
            print(f"    Detected: '{full_text}'")
            print(f"    Characters: {num_chars} (alphanumeric: {num_alphanumeric})")
            print(f"    Avg Confidence: {avg_confidence:.1f}%")
            print(f"    Checks: length={has_sufficient_length}, conf={has_sufficient_confidence}, alnum={has_alphanumeric}")
            print(f"    Result: {'PASS' if passes_verification else 'FAIL'}")

        return passes_verification, text_info

    def correct_orientation(self, roi):
        """
        Stage 8: Detect and correct the orientation of the whiteboard.
        Tests OCR at 0°, 90°, 180°, 270° and selects the best orientation.

        Args:
            roi: Extracted whiteboard ROI

        Returns:
            corrected_roi: ROI rotated to correct orientation
            orientation_info: Dictionary with rotation details
        """
        try:
            import pytesseract
        except ImportError:
            if self.debug:
                print("  Warning: pytesseract not available. Skipping orientation correction.")
            return roi, {'skipped': True, 'best_angle': 0}

        best_confidence = -1
        best_angle = 0
        best_text = ""
        orientation_results = {}

        if self.debug:
            print(f"  Testing orientations: {self.test_orientations}")

        for angle in self.test_orientations:
            # Rotate the image
            if angle == 0:
                rotated = roi.copy()
            elif angle == 90:
                rotated = cv2.rotate(roi, cv2.ROTATE_90_CLOCKWISE)
            elif angle == 180:
                rotated = cv2.rotate(roi, cv2.ROTATE_180)
            elif angle == 270:
                rotated = cv2.rotate(roi, cv2.ROTATE_90_COUNTERCLOCKWISE)
            else:
                rotated = roi.copy()

            # Preprocess for OCR
            gray = cv2.cvtColor(rotated, cv2.COLOR_BGR2GRAY)
            binary = cv2.adaptiveThreshold(
                gray, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
                cv2.THRESH_BINARY, 11, 2
            )
            denoised = cv2.fastNlMeansDenoising(binary, None, 10, 7, 21)

            # Run OCR
            try:
                ocr_data = pytesseract.image_to_data(
                    denoised,
                    output_type=pytesseract.Output.DICT,
                    config='--psm 6'
                )

                # Calculate average confidence for this orientation
                confidences = [int(conf) for conf in ocr_data['conf'] if int(conf) > 0]
                avg_conf = sum(confidences) / len(confidences) if confidences else 0

                # Extract text
                text_parts = [ocr_data['text'][i].strip() for i, conf in enumerate(ocr_data['conf'])
                             if int(conf) > 0 and ocr_data['text'][i].strip()]
                detected_text = ' '.join(text_parts)

                orientation_results[angle] = {
                    'confidence': avg_conf,
                    'text': detected_text,
                    'num_chars': len(detected_text.replace(' ', ''))
                }

                if self.debug:
                    print(f"    {angle:3d}°: confidence={avg_conf:5.1f}%, text='{detected_text}'")

                # Update best orientation based on confidence
                if avg_conf > best_confidence:
                    best_confidence = avg_conf
                    best_angle = angle
                    best_text = detected_text

            except Exception as e:
                if self.debug:
                    print(f"    {angle:3d}°: OCR failed - {e}")
                orientation_results[angle] = {
                    'confidence': 0,
                    'text': '',
                    'error': str(e)
                }

        # Apply the best rotation
        if best_angle == 0:
            corrected_roi = roi.copy()
        elif best_angle == 90:
            corrected_roi = cv2.rotate(roi, cv2.ROTATE_90_CLOCKWISE)
        elif best_angle == 180:
            corrected_roi = cv2.rotate(roi, cv2.ROTATE_180)
        elif best_angle == 270:
            corrected_roi = cv2.rotate(roi, cv2.ROTATE_90_COUNTERCLOCKWISE)
        else:
            corrected_roi = roi.copy()

        orientation_info = {
            'best_angle': best_angle,
            'best_confidence': best_confidence,
            'best_text': best_text,
            'all_results': orientation_results
        }

        if self.debug:
            print(f"  Selected orientation: {best_angle}° (confidence: {best_confidence:.1f}%)")
            print(f"  Best text: '{best_text}'")

        return corrected_roi, orientation_info

    def visualize_detection_stages(self, frame, validated_candidates_with_edges=None, extracted_rois=None):
        """
        Visualize the results of each detection stage for debugging.

        Args:
            frame: Original frame
            validated_candidates_with_edges: List of candidates that passed edge density
            extracted_rois: List of extracted and corrected ROIs
        """
        # Determine grid size based on whether we have ROIs
        if extracted_rois:
            fig, axes = plt.subplots(3, 3, figsize=(18, 16))
        else:
            fig, axes = plt.subplots(2, 3, figsize=(18, 12))

        # Stage 1: White mask
        axes[0, 0].imshow(self.white_mask, cmap='gray')
        axes[0, 0].set_title(f'Stage 1: White Regions ({len(self.all_contours)} found)')
        axes[0, 0].axis('off')

        # Stage 1: Contours on original
        frame_contours = frame.copy()
        cv2.drawContours(frame_contours, self.all_contours, -1, (0, 255, 0), 2)
        axes[0, 1].imshow(cv2.cvtColor(frame_contours, cv2.COLOR_BGR2RGB))
        axes[0, 1].set_title('Stage 1: All White Contours')
        axes[0, 1].axis('off')

        # Stage 2: Filtered by shape
        frame_filtered = frame.copy()
        cv2.drawContours(frame_filtered, self.filtered_contours, -1, (0, 255, 0), 3)
        # Draw bounding boxes
        for rect in self.candidate_boxes:
            box = cv2.boxPoints(rect)
            box = np.intp(box)
            cv2.drawContours(frame_filtered, [box], 0, (255, 0, 0), 2)
        axes[0, 2].imshow(cv2.cvtColor(frame_filtered, cv2.COLOR_BGR2RGB))
        axes[0, 2].set_title(f'Stage 2: Shape Filtered ({len(self.filtered_contours)} candidates)')
        axes[0, 2].axis('off')

        # Stage 3: Geometry validated (before edge density)
        frame_geometry = frame.copy()
        if self.edge_density_results:
            for result in self.edge_density_results:
                contour = result['contour']
                cv2.drawContours(frame_geometry, [contour], -1, (255, 165, 0), 3)  # Orange for geometry pass
        axes[1, 0].imshow(cv2.cvtColor(frame_geometry, cv2.COLOR_BGR2RGB))
        axes[1, 0].set_title(f'Stage 3: Geometry Validated ({len(self.edge_density_results)} candidates)')
        axes[1, 0].axis('off')

        # Stage 4: Edge density visualization (combined edges)
        if self.edge_density_results:
            # Create a combined edge map showing all candidates
            combined_edges = np.zeros(frame.shape[:2], dtype=np.uint8)
            for result in self.edge_density_results:
                edges_img = result['edge_info']['edges_image']
                combined_edges = cv2.bitwise_or(combined_edges, edges_img)
            axes[1, 1].imshow(combined_edges, cmap='gray')
            axes[1, 1].set_title('Stage 4: Edge Detection in Candidates')
        else:
            axes[1, 1].imshow(np.zeros(frame.shape[:2]), cmap='gray')
            axes[1, 1].set_title('Stage 4: No Candidates')
        axes[1, 1].axis('off')

        # Stage 4: Final candidates after edge density filtering
        frame_final = frame.copy()
        if validated_candidates_with_edges:
            for candidate in validated_candidates_with_edges:
                contour = candidate[0]
                edge_density = candidate[3]  # edge_density value
                # Draw in bright green for final candidates
                cv2.drawContours(frame_final, [contour], -1, (0, 255, 0), 4)

                # Add edge density text
                M = cv2.moments(contour)
                if M["m00"] != 0:
                    cX = int(M["m10"] / M["m00"])
                    cY = int(M["m01"] / M["m00"])
                    cv2.putText(frame_final, f"{edge_density*100:.1f}%",
                               (cX-50, cY), cv2.FONT_HERSHEY_SIMPLEX,
                               1.5, (0, 255, 0), 3)
        axes[1, 2].imshow(cv2.cvtColor(frame_final, cv2.COLOR_BGR2RGB))
        axes[1, 2].set_title(f'Stage 4: Edge Density Filtered ({len(validated_candidates_with_edges) if validated_candidates_with_edges else 0} final)')
        axes[1, 2].axis('off')

        # Stage 7: Extracted ROIs (if available)
        if extracted_rois:
            for idx, roi_data in enumerate(extracted_rois[:3]):  # Show up to 3 ROIs
                roi = roi_data['roi']
                roi_info = roi_data['roi_info']

                axes[2, idx].imshow(cv2.cvtColor(roi, cv2.COLOR_BGR2RGB))
                title = f"Stage 7: Extracted ROI {idx+1}\n"
                title += f"Method: {roi_info['method']}\n"
                title += f"Shape: {roi_info['roi_shape'][:2]}"
                axes[2, idx].set_title(title, fontsize=10)
                axes[2, idx].axis('off')

            # Hide unused ROI slots
            for idx in range(len(extracted_rois), 3):
                axes[2, idx].axis('off')

        plt.tight_layout()
        plt.show()

    def process_frame(self, frame):
        """
        Main processing pipeline: detect whiteboard in a single frame.
        Implements Stages 1-5, 7-8.

        Args:
            frame: Input BGR frame from video

        Returns:
            results: List of detection results
                    Each result is a dictionary containing:
                    - 'contour': The whiteboard contour
                    - 'rect': minAreaRect
                    - 'validation_info': Geometry validation details
                    - 'edge_density': Edge density value
                    - 'edge_info': Edge detection details
                    - 'roi': Extracted, orientation-corrected whiteboard image
                    - 'roi_info': ROI extraction details
                    - 'orientation_info': Orientation correction details
                    - 'text_info': OCR verification results
        """
        if self.debug:
            print("\n" + "="*60)
            print("WHITEBOARD DETECTION PIPELINE")
            print("="*60)

        # Stage 1: Detect white regions
        white_mask, contours = self.detect_white_regions(frame)

        if len(contours) == 0:
            if self.debug:
                print("No white regions found!")
            return []

        # Stage 2: Filter by shape
        shape_candidates = self.filter_by_shape(frame, contours)

        if len(shape_candidates) == 0:
            if self.debug:
                print("No candidates passed shape filtering!")
            return []

        # Stage 3: Validate geometry
        geometry_validated = []
        for contour, rect in shape_candidates:
            is_valid, validation_info = self.validate_shape_geometry(frame, contour, rect)
            if is_valid:
                geometry_validated.append((contour, rect, validation_info))

        if self.debug:
            print(f"\nStage 3: {len(geometry_validated)} candidates passed geometry validation")

        if len(geometry_validated) == 0:
            if self.debug:
                print("No candidates passed geometry validation!")
            return []

        # Stage 4: Edge density analysis
        if self.debug:
            print(f"\n{'='*60}")
            print("STAGE 4: EDGE DENSITY ANALYSIS")
            print(f"{'='*60}")

        edge_density_validated = []
        self.edge_density_results = []  # Reset for visualization

        for contour, rect, validation_info in geometry_validated:
            edge_density, edge_info = self.calculate_edge_density(frame, contour, rect)

            # Store for visualization (all candidates that reached this stage)
            self.edge_density_results.append({
                'contour': contour,
                'rect': rect,
                'validation_info': validation_info,
                'edge_density': edge_density,
                'edge_info': edge_info
            })

            # Filter by edge density threshold
            if edge_info['passes_threshold']:
                edge_density_validated.append((contour, rect, validation_info, edge_density, edge_info))

        if self.debug:
            print(f"\nStage 4: {len(edge_density_validated)} candidates passed edge density filtering")

        if len(edge_density_validated) == 0:
            if self.debug:
                print("No candidates passed edge density filtering!")
                self.visualize_detection_stages(frame, [])
            return []

        # Stage 7: Extract and perspective-correct ROIs
        if self.debug:
            print(f"\n{'='*60}")
            print("STAGE 7: PERSPECTIVE CORRECTION & ROI EXTRACTION")
            print(f"{'='*60}")

        extracted_rois = []
        self.extracted_rois = []  # Reset for visualization

        for contour, rect, validation_info, edge_density, edge_info in edge_density_validated:
            # Extract the whiteboard ROI
            roi, roi_info = self.extract_whiteboard_roi(frame, contour, rect, validation_info)

            extracted_rois.append((contour, rect, validation_info, edge_density, edge_info, roi, roi_info))

        if self.debug:
            print(f"\nStage 7: Successfully extracted {len(extracted_rois)} whiteboard ROI(s)")

        if len(extracted_rois) == 0:
            if self.debug:
                self.visualize_detection_stages(frame, edge_density_validated, [])
            return []

        # Stage 8: Orientation Correction (before text verification for better accuracy)
        if self.debug:
            print(f"\n{'='*60}")
            print("STAGE 8: ORIENTATION CORRECTION")
            print(f"{'='*60}")

        orientation_corrected = []

        for contour, rect, validation_info, edge_density, edge_info, roi, roi_info in extracted_rois:
            # Correct the orientation
            corrected_roi, orientation_info = self.correct_orientation(roi)

            # Store the corrected ROI for visualization
            self.extracted_rois.append({
                'roi': corrected_roi,
                'roi_info': roi_info
            })

            orientation_corrected.append((contour, rect, validation_info, edge_density, edge_info, corrected_roi, roi_info, orientation_info))

        if self.debug:
            print(f"\nStage 8: Corrected orientation for {len(orientation_corrected)} ROI(s)")

        # Stage 5: Text Pattern Verification (done after orientation correction)
        if self.debug:
            print(f"\n{'='*60}")
            print("STAGE 5: TEXT PATTERN VERIFICATION")
            print(f"{'='*60}")

        text_verified = []

        for contour, rect, validation_info, edge_density, edge_info, roi, roi_info, orientation_info in orientation_corrected:
            # Verify text presence in the corrected ROI
            has_text, text_info = self.verify_text_presence(roi)

            # Only keep candidates with verified text
            if has_text:
                # Package all information together
                result = {
                    'contour': contour,
                    'rect': rect,
                    'validation_info': validation_info,
                    'edge_density': edge_density,
                    'edge_info': edge_info,
                    'roi': roi,  # This is the orientation-corrected ROI
                    'roi_info': roi_info,
                    'orientation_info': orientation_info,
                    'text_info': text_info
                }

                text_verified.append(result)

        if self.debug:
            print(f"\nStage 5: {len(text_verified)} candidate(s) passed text verification")
            # Visualize with orientation-corrected ROIs
            self.visualize_detection_stages(frame, edge_density_validated, self.extracted_rois)

        return text_verified


class VideoFrameExtractor:
    """
    Extract frames from .360 videos for whiteboard ID detection.
    Now integrated with WhiteboardDetector.
    """

    def __init__(self, video_path, output_dir="extracted_frames", debug_detection=False):
        """
        Initialize the frame extractor.

        Args:
            video_path: Path to the .360 video file
            output_dir: Directory to save extracted frames
            debug_detection: Enable debug mode for whiteboard detection
        """
        self.video_path = video_path
        self.output_dir = output_dir
        self.frames = []
        self.whiteboard_detector = WhiteboardDetector(debug=debug_detection)

        # Create output directory if it doesn't exist
        Path(self.output_dir).mkdir(parents=True, exist_ok=True)

    def extract_frames(self, duration_seconds=100, fps_sample=10):
        """
        Extract frames from the first N seconds of the video.

        Args:
            duration_seconds: How many seconds to extract from (default: 100)
            fps_sample: Sample rate - extract 1 frame per N seconds (default: 10)

        Returns:
            List of extracted frame paths
        """
        cap = cv2.VideoCapture(self.video_path)
        print(f"Sample rate is:{fps_sample}")

        if not cap.isOpened():
            raise ValueError(f"Error: Could not open video file {self.video_path}. "
                           f"The file exists but OpenCV cannot read it. "
                           f"Try installing: !apt-get install -y ffmpeg")

        # Get video properties
        video_fps = cap.get(cv2.CAP_PROP_FPS)
        total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
        height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

        print(f"Video Properties:")
        print(f"  - FPS: {video_fps}")
        print(f"  - Total Frames: {total_frames}")
        print(f"  - Resolution: {width}x{height}")
        print(f"  - Duration: {total_frames/video_fps:.2f} seconds")

        # Calculate frame interval for sampling
        frame_interval = max(1, int(video_fps * fps_sample))
        max_frame = min(int(video_fps * duration_seconds), total_frames)

        print(f"\nExtracting frames:")
        print(f"  - Sampling every {frame_interval} frames (≈{fps_sample}s intervals)")
        print(f"  - Max frame index: {max_frame}")

        frame_count = 0
        extracted_count = 0
        extracted_paths = []
        consecutive_failures = 0
        max_consecutive_failures = 10  # Stop if we can't read 10 frames in a row

        while cap.isOpened() and frame_count < max_frame:
            ret, frame = cap.read()

            # Handle read failures - try to skip problematic frames
            if not ret:
                consecutive_failures += 1
                print(f"  Warning: Failed to read frame {frame_count} (consecutive failures: {consecutive_failures})")

                # If too many consecutive failures, stop processing
                if consecutive_failures >= max_consecutive_failures:
                    print(f"\n*** Too many consecutive failures ({consecutive_failures}). Stopping extraction. ***")
                    print(f"*** Successfully extracted {extracted_count} frames before failure. ***")
                    break

                # Try to skip to next frame
                frame_count += 1
                cap.set(cv2.CAP_PROP_POS_FRAMES, frame_count)
                continue

            # Reset consecutive failures on successful read
            consecutive_failures = 0

            # Validate frame is not empty
            if frame is None or frame.size == 0:
                print(f"  Warning: Frame {frame_count} is empty, skipping")
                frame_count += 1
                continue

            # Extract frame at specified intervals
            if frame_count % frame_interval == 0:
                timestamp = frame_count / video_fps
                frame_filename = f"frame_{extracted_count:03d}_t{timestamp:.2f}s.jpg"
                frame_path = os.path.join(self.output_dir, frame_filename)

                # Save frame
                success = cv2.imwrite(frame_path, frame)
                if success:
                    extracted_paths.append(frame_path)
                    self.frames.append(frame)
                    print(f"  Extracted: {frame_filename}")
                    extracted_count += 1
                else:
                    print(f"  Error: Failed to write {frame_filename}")

            frame_count += 1

        cap.release()

        print(f"\nTotal frames extracted: {extracted_count}")
        return extracted_paths

    def display_frames(self, num_frames=6, figsize=(15, 10)):
        """
        Display extracted frames in a grid for manual review.

        Args:
            num_frames: Number of frames to display (default: 6)
            figsize: Figure size for matplotlib (default: (15, 10))
        """
        if not self.frames:
            print("No frames extracted yet. Run extract_frames() first.")
            return

        num_frames = min(num_frames, len(self.frames))
        cols = 3
        rows = (num_frames + cols - 1) // cols

        fig, axes = plt.subplots(rows, cols, figsize=figsize)

        if rows == 1 and cols == 1:
            axes = np.array([axes])
        else:
            axes = axes.flatten()

        for idx in range(num_frames):
            frame = self.frames[idx]
            frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

            axes[idx].imshow(frame_rgb)
            axes[idx].set_title(f"Frame {idx}")
            axes[idx].axis('off')

        for idx in range(num_frames, len(axes)):
            axes[idx].axis('off')

        plt.tight_layout()
        plt.show()

    def detect_whiteboards_in_frames(self, frame_indices=None):
        """
        Run whiteboard detection on extracted frames.

        Args:
            frame_indices: List of frame indices to process, or None to process all

        Returns:
            detection_results: Dictionary mapping frame_index to detection results
        """
        if not self.frames:
            print("No frames extracted yet. Run extract_frames() first.")
            return {}

        if frame_indices is None:
            frame_indices = range(len(self.frames))

        detection_results = {}

        for idx in frame_indices:
            if idx >= len(self.frames):
                continue

            print(f"\n{'='*60}")
            print(f"Processing Frame {idx}")
            print(f"{'='*60}")

            frame = self.frames[idx]
            candidates = self.whiteboard_detector.process_frame(frame)
            detection_results[idx] = candidates

            print(f"Frame {idx}: Found {len(candidates)} whiteboard candidate(s)")

        return detection_results

    def get_frame_info(self):
        """
        Get information about extracted frames.

        Returns:
            Dictionary with frame information
        """
        if not self.frames:
            return {"message": "No frames extracted yet"}

        return {
            "total_frames": len(self.frames),
            "frame_shape": self.frames[0].shape if self.frames else None,
            "output_directory": self.output_dir
        }


# Example usage function
def process_video(video_path, duration=7, sample_rate=1, display_count=6, debug_detection=False):
    """
    Complete workflow to extract and display frames from a video.

    Args:
        video_path: Path to the .360 video file
        duration: Seconds to extract from start of video (default: 10)
        sample_rate: Extract 1 frame per N seconds (default: 10)
        display_count: Number of frames to display (default: 6)
        debug_detection: Enable whiteboard detection debugging (default: False)
    """
    print(f"Processing video: {video_path}\n")

    # Create extractor instance with whiteboard detection
    extractor = VideoFrameExtractor(video_path, debug_detection=debug_detection)

    # Extract frames
    frame_paths = extractor.extract_frames(
        duration_seconds=duration,
        fps_sample=sample_rate
    )

    # Display frames
    print("\nDisplaying frames for manual review...")
    extractor.display_frames(num_frames=display_count)

    # Print summary
    info = extractor.get_frame_info()
    print(f"\nFrame Info:")
    print(f"  - Total extracted: {info['total_frames']}")
    print(f"  - Frame dimensions: {info['frame_shape']}")
    print(f"  - Saved to: {info['output_directory']}")

    return extractor, frame_paths


# Usage example with whiteboard detection:
extractor, paths = process_video("GS012395.360", duration=7, sample_rate=1, debug_detection=True)
results = extractor.detect_whiteboards_in_frames(frame_indices=[0, 1, 2])  # Test on first 3 frames

Processing video: GS012395.360

Sample rate is:1


ValueError: Error: Could not open video file GS012395.360. The file exists but OpenCV cannot read it. Try installing: !apt-get install -y ffmpeg